In [24]:
# tokens = ['can', 'you', 'please', 'order', 'one', 'large', 'pepperoni', 'pizza', 'and', 'one', 'medium', 'sausage', 'pizza', 'with', 'one', 'sprite', 'and', 'one', 'coke']
# labels = [0,0,0,0,1,2,4,0,0,1,2,4,0,0,1,8,0,1,8]
# tokens=['two', 'pizzes', 'one', 'with', 'pepproni', 'and','one','with','chicken']
# labels=[1,0,1,0,4,0,1,0,4]
tokens=['want','one','pizza', 'with', 'pepproni', 'and','with','large','size','iced','tea']
labels=[0,1,0,0,4,0,0,2,2,8,8]
def combine_tokens(tokens, labels):
    combined_tokens = []
    combined_labels = []
    i = 0

    while i < len(tokens):
        if labels[i] != 0:  # Check for non-None labels
            # Start combining consecutive tokens with the same label
            combined_token = tokens[i]
            current_label = labels[i]
            i += 1

            while i < len(tokens) and labels[i] == current_label:
                combined_token += f"_{tokens[i]}"
                i += 1

            combined_tokens.append(combined_token)
            combined_labels.append(current_label)
        else:
            # Keep tokens with label 0 as is
            combined_tokens.append(tokens[i])
            combined_labels.append(labels[i])
            i += 1

    return combined_tokens, combined_labels

combined_tokens, combined_labels = combine_tokens(tokens, labels)
print(combined_tokens)
print(combined_labels)

['want', 'pizza', 'with', 'pepproni', 'and', 'one', 'with', 'large_size', 'iced_tea']
[0, 0, 0, 4, 0, 1, 0, 2, 8]


In [25]:
label_mapping = {
    0: "None",
    1: "Number",
    2: "SIZE",
    3: "STYLE",
    4: "Topping",
    5: "Quantity",
    6: "NOT_TOPPING",
    7: "NOT_STYLE",
    8: "DRINKTYPE",
    9: "CONTAINERTYPE",
    10: "VOLUME",
}

def extract_order_details(tokens, labels):
    # Step 1: Split into suborders
    suborders = []
    current_suborder = {"tokens": [], "labels": [], "has_number": False, "has_size": False, "has_style": False}
    
    for token, label in zip(tokens, labels):
        if label == 1 and current_suborder["tokens"]:  # New suborder starts at 'Number'
            suborders.append(current_suborder)
            current_suborder = {"tokens": [], "labels": [], "has_number": False, "has_size": False, "has_style": False}
        
        # If the label is SIZE and we already have a size, start a new suborder
        if label == 2 and current_suborder["has_size"] and current_suborder["tokens"]:
            suborders.append(current_suborder)
            current_suborder = {"tokens": [], "labels": [], "has_number": False, "has_size": False, "has_style": False}
        
        # If the label is STYLE and we already have a style, start a new suborder
        if label == 3 and current_suborder["has_style"] and current_suborder["tokens"]:
            suborders.append(current_suborder)
            current_suborder = {"tokens": [], "labels": [], "has_number": False, "has_size": False, "has_style": False}
        
        current_suborder["tokens"].append(token)
        current_suborder["labels"].append(label)

        # Update flags for size and style
        if label == 2:  # SIZE label
            current_suborder["has_size"] = True
        elif label == 3:  # STYLE label
            current_suborder["has_style"] = True
        elif label == 1:  # NUMBER label
            current_suborder["has_number"] = True
    
    # Append the last suborder
    if current_suborder["tokens"]:
        suborders.append(current_suborder)
    
    # Step 2: Add default Number if no Number found
    for suborder in suborders:
        if not suborder["has_number"]:
            suborder["tokens"].insert(0, "one")
            suborder["labels"].insert(0, 1)  # Number label
    
    # Step 3: Classify suborders
    order_details = []
    for suborder in suborders:
        tokens = suborder["tokens"]
        labels = suborder["labels"]
        
        # Determine order type
        if any(label in [8, 9, 10] for label in labels):  # Drink order
            order_type = "DRINKORDER"
        elif any(label in [3, 4, 5, 6, 7] for label in labels):  # Pizza order
            order_type = "PIZZAORDER"
        else:
            continue  # Skip invalid suborders
        
        suborder_details = []
        i = 0
        while i < len(tokens):
            token = tokens[i]
            label = labels[i]
            if label == 5:  # Quantity
                suborder_details.append(
                    f"(Complex_Topping (Quantity {token.upper()}) (Topping {tokens[i + 1].upper()}))"
                )
                i += 1
            elif label in [6, 7]:  # NOT cases (NOT_TOPPING, NOT_STYLE)
                new_label = label_mapping[label][4:]  # Remove "NOT_" from the label string
                suborder_details.append(
                    f"(NOT ({new_label} {token.upper()}))"
                )
            elif label != 0:
                # General case for non-zero labels
                suborder_details.append(f"({label_mapping[label]} {token.upper()})")
            i += 1
        
        order_details.append(f"({order_type} {' '.join(suborder_details)})")
    
    # Step 4: Combine into final order
    return f"(ORDER {' '.join(order_details)})"


# Extract order details
order_details_reverse = extract_order_details(combined_tokens, labels)

# Output the order details
print("Order Details (Reverse Parsing):", order_details_reverse)

Order Details (Reverse Parsing): (ORDER (PIZZAORDER (Number ONE) (Topping PEPPRONI)) (DRINKORDER (Number ONE) (SIZE LARGE_SIZE) (DRINKTYPE ICED_TEA)))
